In [3]:
import os
import requests

In [5]:
pdf_path=input("Give the pdf name:")

if not os.path.exists(pdf_path):
    print(f"[INFO] File doesn't exist,downloading...")

    url=input("Give the download url")

    #File name to save the downloaded file
    filename=pdf_path

    #Send a get request was successful
    response=requests.get(url)

    #check if the request was successful
    if response.status_code==200:
        #Open the file and save it
        with open(filename,"wb") as file:
            file.write(response.content)
        print(f"[INFO] the file has been downloaded as {filename}")
    else:
        print(f"[INFO] Failed to download the file. Status code: {response.status_code}")
else:
    print(f"File {pdf_path} exists.")

Give the pdf name: human-nutrition-text.pdf


File human-nutrition-text.pdf exists.


In [6]:
pip install pymupdf


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import fitz
from tqdm.auto import tqdm

def text_formatter(text: str) -> str:
    #Performs formatting
    cleaned_text=text.replace("\n", " ").strip()

    return cleaned_text

def open_and_read_pdf(pdf_path: str )-> list[dict]:
    doc=fitz.open(pdf_path)
    pages_and_texts=[]
    for page_number,page in tqdm(enumerate(doc)):
        text=page.get_text()
        text=text_formatter(text=text)
        pages_and_texts.append({"page_number":page_number,
                                "page_char_count":len(text),
                                "page_sentence_count":len(text.split(". ")),
                                "page_word_count":len(text.split(" ")),
                                "page_token_count": len(text)/4,
                                "text":text})# 1 token = ~4charcters
    return pages_and_texts
pages_and_texts=open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[100]


c:\users\kisho\appdata\local\programs\python\python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
1208it [00:03, 349.17it/s]


{'page_number': 100,
 'page_char_count': 629,
 'page_sentence_count': 4,
 'page_word_count': 109,
 'page_token_count': 157.25,
 'text': 'Digestive  system  without  labels by  Mariana  Ruiz / Public  Domain  Knowing how to maintain the balance of friendly bacteria in your  intestines through proper diet can promote overall health. Recent  scientific studies have shown that probiotic supplements positively  affect intestinal microbial flora, which in turn positively affect  immune system function. As good nutrition is known to influence  immunity, there is great interest in using probiotic foods and other  immune-system-friendly foods as a way to prevent illness. In this  chapter we will explore not only immune system function, but also  Introduction  |  59'}

In [8]:
import pandas as pd
df=pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_sentence_count,page_word_count,page_token_count,text
0,0,29,1,4,7.25,Human Nutrition: 2020 Edition
1,1,0,1,1,0.00,
2,2,320,1,54,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...
3,3,212,1,32,53.00,Human Nutrition: 2020 Edition by University of...
4,4,797,2,145,199.25,Contents Preface University of Hawai‘i at Mā...


In [9]:
df.describe()

,page_number,page_char_count,page_sentence_count,page_word_count,page_token_count
count,1208.00000,1208.000000,1208.000000,1208.000000,1208.000000
mean,603.50000,1148.004139,9.972682,198.299669,287.001035
std,348.86387,560.382275,6.187226,95.759336,140.095569
min,0.00000,0.000000,1.000000,1.000000,0.000000
25%,301.75000,762.000000,4.000000,134.000000,190.500000
50%,603.50000,1231.500000,10.000000,214.500000,307.875000
75%,905.25000,1603.500000,14.000000,271.000000,400.875000
max,1207.00000,2308.000000,32.000000,429.000000,577.000000


In [10]:
from spacy.lang.en import English

In [11]:
nlp=English()

nlp.add_pipe("sentencizer")

doc=nlp("This is a sentence. This another sentence. I like elephants.")
assert len(list(doc.sents))==3

list(doc.sents)

[This is a sentence., This another sentence., I like elephants.]

In [12]:
for item in tqdm(pages_and_texts):
    item["sentences"]=list(nlp(item["text"]).sents)
    #Datatype conversion
    item["sentences"]=[str(sentence) for sentence in item["sentences"]]
    #Count the sentences
    item["page_sentence_count_spacy"]=len(item["sentences"])

100%|█████████████████████████████████████████████████████████████████████████████| 1208/1208 [00:06<00:00, 199.67it/s]


In [13]:
pages_and_texts[100]

{'page_number': 100,
 'page_char_count': 629,
 'page_sentence_count': 4,
 'page_word_count': 109,
 'page_token_count': 157.25,
 'text': 'Digestive  system  without  labels by  Mariana  Ruiz / Public  Domain  Knowing how to maintain the balance of friendly bacteria in your  intestines through proper diet can promote overall health. Recent  scientific studies have shown that probiotic supplements positively  affect intestinal microbial flora, which in turn positively affect  immune system function. As good nutrition is known to influence  immunity, there is great interest in using probiotic foods and other  immune-system-friendly foods as a way to prevent illness. In this  chapter we will explore not only immune system function, but also  Introduction  |  59',
 'sentences': ['Digestive  system  without  labels by  Mariana  Ruiz / Public  Domain  Knowing how to maintain the balance of friendly bacteria in your  intestines through proper diet can promote overall health.',
  'Recent  scient

In [14]:
df=pd.DataFrame(pages_and_texts)
df.describe()

,page_number,page_char_count,page_sentence_count,page_word_count,page_token_count,page_sentence_count_spacy
count,1208.00000,1208.000000,1208.000000,1208.000000,1208.000000,1208.000000
mean,603.50000,1148.004139,9.972682,198.299669,287.001035,10.319536
std,348.86387,560.382275,6.187226,95.759336,140.095569,6.300843
min,0.00000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,301.75000,762.000000,4.000000,134.000000,190.500000,5.000000
50%,603.50000,1231.500000,10.000000,214.500000,307.875000,10.000000
75%,905.25000,1603.500000,14.000000,271.000000,400.875000,15.000000
max,1207.00000,2308.000000,32.000000,429.000000,577.000000,28.000000


In [15]:
#Chunck size
num_sentence_chunk_size=10

#Create a recursive function to split sentences into chunks
def split_list(input_list:list,slice_size:int)->list[list[str]]:
    return [input_list[i:i+slice_size] for i in range(0,len(input_list),slice_size)]


In [16]:
#Loop through pages and texts and split sentences into chunks
for item in tqdm(pages_and_texts):
    item["sentence_chunks"]=split_list(item["sentences"],num_sentence_chunk_size)

    item["num_chunks"]=len(item["sentence_chunks"])


100%|██████████████████████████████████████████████████████████████████████████| 1208/1208 [00:00<00:00, 158236.08it/s]


In [17]:
import random
random.sample(pages_and_texts,k=1)

[{'page_number': 873,
  'page_char_count': 1418,
  'page_sentence_count': 8,
  'page_word_count': 228,
  'page_token_count': 354.5,
  'text': 'Breastfeeding. These steps include having a written policy  on breastfeeding communicated to health-care staff on a  routine basis, informing all new mothers about the benefits  and management of breastfeeding, showing new mothers  how to breastfeed their infants, and how to maintain  lactation, and giving newborns no food or drink other than  breast milk, unless medically indicated. Since the BFHI  began, more than fifteen thousand facilities in 134  countries, from Benin to Bangladesh, have been deemed  “baby friendly.” As a result, more mothers are breastfeeding  their newborns and infant health has improved, in both the  developed world and in developing nations.22 For  information on the Baby-Friendly Initiative in the US, see:  https://www.babyfriendlyusa.org.  Barriers to Breastfeeding  Although breast milk is ideal for almost all infants

In [18]:
df=pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_sentence_count,page_word_count,page_token_count,page_sentence_count_spacy,num_chunks
count,1208.00,1208.00,1208.00,1208.00,1208.00,1208.00,1208.00
mean,603.50,1148.00,9.97,198.30,287.00,10.32,1.53
std,348.86,560.38,6.19,95.76,140.10,6.30,0.64
min,0.00,0.00,1.00,1.00,0.00,0.00,0.00
25%,301.75,762.00,4.00,134.00,190.50,5.00,1.00
50%,603.50,1231.50,10.00,214.50,307.88,10.00,1.00
75%,905.25,1603.50,14.00,271.00,400.88,15.00,2.00
max,1207.00,2308.00,32.00,429.00,577.00,28.00,3.00


In [19]:
import re
pages_and_chunks=[]
for item in tqdm(pages_and_texts):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict={}
        chunk_dict["page_number"]=item["page_number"]

        joined_sentence="".join(sentence_chunk).replace("  "," ").strip()
        joined_sentence=re.sub(r'\.([A-Z])',r'. \1',joined_sentence)


        chunk_dict["sentence_chunk"]=joined_sentence

        chunk_dict["chunk_char_count"]=len(joined_sentence)
        chunk_dict["chunk_word_count"]=len([word for word in joined_sentence.split(" ")])
        chunk_dict["chunk_token_count"]=len(joined_sentence)/4 #1 token ~ 4 chars

        pages_and_chunks.append(chunk_dict)

len(pages_and_chunks)

100%|███████████████████████████████████████████████████████████████████████████| 1208/1208 [00:00<00:00, 12575.13it/s]


1843

In [20]:
random.sample(pages_and_chunks,k=1)

[{'page_number': 977,
  'sentence_chunk': 'http://www.heart.org/HEARTORG/ HealthyLiving/PhysicalActivity/FitnessBasics/ American-Heart-Association-Recommendations-for- 936 | The Essential Elements of Physical Fitness',
  'chunk_char_count': 173,
  'chunk_word_count': 11,
  'chunk_token_count': 43.25}]

In [21]:
df=pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,1843.00,1843.00,1843.00,1843.00
mean,624.38,734.44,112.33,183.61
std,347.79,447.54,71.22,111.89
min,0.00,12.00,3.00,3.00
25%,321.50,315.00,44.00,78.75
50%,627.00,746.00,114.00,186.50
75%,931.00,1118.50,173.00,279.62
max,1207.00,1831.00,297.00,457.75


In [22]:
min_token_len=30
pages_and_chunks_over_min_token_len=df[df["chunk_token_count"] > min_token_len].to_dict(orient="records")

In [23]:
random.sample(pages_and_chunks_over_min_token_len,k=1)
len(pages_and_chunks_over_min_token_len)

1680

In [25]:
from sentence_transformers import SentenceTransformer
embedding_model=SentenceTransformer(model_name_or_path="all-mpnet-base-v2",device="cpu")

#Loading embedding model

In [26]:
for item in tqdm(pages_and_chunks_over_min_token_len):
    item["embedding"]=embedding_model.encode(item["sentence_chunk"])

#Embedding the pdf

100%|██████████████████████████████████████████████████████████████████████████████| 1680/1680 [22:54<00:00,  1.22it/s]


In [27]:
pages_and_chunks_embedding_df=pd.DataFrame(pages_and_chunks_over_min_token_len)
pages_and_embedding_path="pages_and_chunks_embedding_df.csv"
pages_and_chunks_embedding_df.to_csv(pages_and_embedding_path,index=False)

#Saving the embedding of pdf in a file as csv

In [28]:
import torch
import numpy as np
import pandas as pd

device="cuda" if torch.cuda.is_available() else "cpu"

text_chunks_embedding=pd.read_csv(pages_and_embedding_path)

text_chunks_embedding["embedding"]=text_chunks_embedding["embedding"].apply(lambda x: np.fromstring(x.strip("[]"),sep=" "))

embedding=torch.tensor(np.stack(text_chunks_embedding["embedding"].tolist(),axis=0),dtype=torch.float32).to(device)

pages_and_chunks=text_chunks_embedding.to_dict(orient="records")

text_chunks_embedding

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,2,Human Nutrition: 2020 Edition UNIVERSITY OF HA...,308,42,77.00,"[0.0674242899, 0.090228036, -0.00509548606, -0..."
1,3,Human Nutrition: 2020 Edition by University of...,210,30,52.50,"[0.0552156381, 0.0592138283, -0.0166167859, -0..."
2,4,Contents Preface University of Hawai‘i at Māno...,766,114,191.50,"[0.0279801637, 0.0339813344, -0.0206427164, 0...."
3,5,Lifestyles and Nutrition University of Hawai‘i...,941,142,235.25,"[0.0682566762, 0.0381274372, -0.00846859161, -..."
4,6,The Cardiovascular System University of Hawai‘...,998,152,249.50,"[0.033026427, -0.0084977299, 0.00957151968, -0..."
...,...,...,...,...,...,...
1675,1205,Flashcard Images Note: Most images in the flas...,1305,176,326.25,"[0.018562289, -0.0164279304, -0.0127046388, -0..."
1676,1205,Hazard Analysis Critical Control Points reused...,375,51,93.75,"[0.0334722437, -0.0570441037, 0.0151489321, -0..."
1677,1206,ShareAlike 11. Organs reused “Pancreas Organ A...,1286,173,321.50,"[0.0770513862, 0.00978544727, -0.0121817011, 0..."
1678,1206,Sucrose reused “Figure 03 02 05” by OpenStax B...,410,59,102.50,"[0.103045143, -0.0164702218, 0.00826840382, 0...."


In [29]:
embedding.shape

torch.Size([1680, 768])

In [30]:
from sentence_transformers import util

In [31]:
def retrieval(embedding=embedding,embedding_model=embedding_model):
      query=input("What is your query?:")

      query_embedding=embedding_model.encode(query,convert_to_tensor=True).to(device)

      dot_scores=util.dot_score(a=embedding,b=query_embedding).reshape(-1,)

      top_k_chunks=torch.topk(dot_scores,k=5)

      return top_k_chunks,query

In [32]:
def print_retrieved_text(top_k_chunks,text_chunks_embedding=text_chunks_embedding):
    for idx in top_k_chunks.indices.tolist():
        print("page number:",text_chunks_embedding["page_number"][idx])
        print(idx,text_chunks_embedding["sentence_chunk"][idx])

        print("\n")

In [37]:
top_k_chunks,query=retrieval(embedding,embedding_model)
print_retrieved_text(top_k_chunks=top_k_chunks)

What is your query?: give 13 points about macronutrients


page number: 46
42 Macronutrients Nutrients that are needed in large amounts are called macronutrients. There are three classes of macronutrients: carbohydrates, lipids, and proteins. These can be metabolically processed into cellular energy. The energy from macronutrients comes from their chemical bonds. This chemical energy is converted into cellular energy that is then utilized to perform work, allowing our bodies to conduct their basic functions. A unit of measurement of food energy is the calorie. On nutrition food labels the amount given for “calories” is actually equivalent to each calorie multiplied by one thousand. A kilocalorie (one thousand calories, denoted with a small “c”) is synonymous with the “Calorie” (with a capital “C”) on nutrition food labels. Water is also a macronutrient in the sense that you require a large amount of it, but unlike the other macronutrients, it does not yield calories. Carbohydrates Carbohydrates are molecules composed of carbon, hydrogen, and o

In [38]:
def concatenating_text(top_k_chunks,text_chunks_embedding=text_chunks_embedding):
    retrieved_text=[]
    for idx in top_k_chunks.indices.tolist():
        retrieved_text.append(text_chunks_embedding["sentence_chunk"][idx])
    context="\n".join(retrieved_text)
    return context

In [39]:
def prompt(query,top_k_chunks,text_chunks_embedding=text_chunks_embedding):
  context=concatenating_text(top_k_chunks,text_chunks_embedding)
  input_text=f"Context: {context}\n\nQuestion: {query}\nAnswer:"
  return input_text

In [41]:
import requests

def generate_response(query, top_k_chunks):
    context = concatenating_text(top_k_chunks, text_chunks_embedding=text_chunks_embedding)
    input_text = f"Context: {context}\n\nQuestion: {query}\nAnswer:"

    response = requests.post("http://localhost:11434/api/generate", json={
        "model": "mistral",
        "prompt": input_text,
        "stream": False
    })

    if response.status_code == 200:
        return response.json().get("response", "").strip()
    else:
        return f"Error from local model: {response.status_code} - {response.text}"
generate_response(query,top_k_chunks)

'1. Macronutrients are substances required by the body in large amounts to perform its basic functions.\n2. The three classes of macronutrients are carbohydrates, lipids (fats), and proteins.\n3. Carbohydrates are molecules composed of carbon, hydrogen, and oxygen. They provide energy for the body and make up a significant portion of many foods.\n4. Lipids (fats) are another source of energy for the body, providing approximately 9 kilocalories per gram. They also serve as insulation, protect organs, and help absorb fat-soluble vitamins.\n5. Proteins are macromolecules composed of chains of subunits called amino acids. They provide structure to bones, muscles, and skin, and play a role in conducting most chemical reactions in the body.\n6. Each gram of protein provides approximately 4 kilocalories of energy.\n7. Water is also a macronutrient that must be obtained from our diet since the human body does not produce it. It makes up more than 60% of total body weight and is essential for t